In [ ]:
#это блок ТГ, можно не запускать

In [1]:
!pip install -U openai -q
!pip install pytelegrambotapi -q
!pip install -U deep-translator -q
!pip install langchain tiktoken -q
!pip install pydub -q
!pip install git+https://github.com/openai/whisper.git -q
!sudo apt update && sudo apt install ffmpeg


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.9/224.9 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.
tensorflow-probability 0.22.0 requires typing-extensions<4.6.0, but you have typing-extensions 4.9.0 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 4.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 798.0/798.0 kB 10.

#Запускаем

In [1]:
!pip install langchain
!pip install torch
!pip install sentence_transformers
!pip install faiss-cpu
!pip install huggingface-hub
!pip install pypdf
!pip -q install accelerate
!pip install llama-cpp-python
!pip -q install git+https://github.com/huggingface/transformers
!pip install fpdf


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 798.0/798.0 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.6/216.6 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.4/48.4 kB 632.5 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 2.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 13.4 MB/s eta 0:00:00
  Created wheel for sentence_transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=2eeb4175a4960f7c3228dc4c8c36dc16ef003b92c9ef476d913ab7ddda7739a4
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence_transformers
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 52.1 M

In [ ]:
#Эту ячейку ниже можно не запускать ТГ не вышло(увидите по коду ниже почему)

In [ ]:
from openai import OpenAI
import telebot
import matplotlib.pyplot as plt
from deep_translator import GoogleTranslator
from PIL import Image
import io
import base64
from langchain.chat_models import ChatOpenAI
import os

In [ ]:
#Запускаем ниже остальное

In [3]:
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.llms import LlamaCpp
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.document_loaders import PyPDFDirectoryLoader

In [4]:
#connect to google drive для загрузки модели с гугл диска
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


Нужно скачать файл модели вот отсюда https://huggingface.co/TheBloke/Mistral-7B-Instruct-v0.1-GGUF/tree/main , подойдет любая с форматом guff/ но чем больше размер тем дольше обработка. В ячейки ниже при параметрах  n_ctx=4096 и

```
text_splitter = RecursiveCharacterTextSplitter(chunk_size=5000, chunk_overlap=100)
text_chunks = text_splitter.split_documents(data)
```


 средняя время обработки, например  уголовного кодекса 4-7 минут(300 страниц) одного вопроса, но маленькие 1-5 страниц (пол минуты - 1.5минуты), но любой первый вопрос он будет обрабатывать медленее чем обычно

1.   А при n_ctx=4096, chunk_size=10000, chunk_overlap=2000 больше часа. Я остановил спустя 1:12/ И не знаю за сколько он справится. Как интересный пример, оставлю на ночь и посмотрю. Но это уже не в проекте.



In [5]:
#Import Model
llm = LlamaCpp(
    streaming = True,
    model_path="/content/drive/MyDrive/mistral-7b-instruct-v0.1.Q4_K_S.gguf",
    temperature=0.75,
    top_p=1,
    verbose=True,
    n_ctx=4096,

)

AVX = 1 | AVX_VNNI = 0 | AVX2 = 1 | AVX512 = 1 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 0 | SSE3 = 1 | SSSE3 = 1 | VSX = 0 | 


In [7]:
print(llm)

LlamaCpp
Params: {'model_path': '/content/drive/MyDrive/mistral-7b-instruct-v0.1.Q4_K_S.gguf', 'suffix': None, 'max_tokens': 256, 'temperature': 0.75, 'top_p': 1.0, 'logprobs': None, 'echo': False, 'stop_sequences': [], 'repeat_penalty': 1.1, 'top_k': 40}


#Установка библиотек для распознования голосов

In [11]:
!pip install -q git+https://github.com/openai/whisper.git > /dev/null
!pip install -q git+https://github.com/pyannote/pyannote-audio > /dev/null

import whisper
import datetime

import subprocess

import torch
import pyannote.audio
from pyannote.audio.pipelines.speaker_verification import PretrainedSpeakerEmbedding
embedding_model = PretrainedSpeakerEmbedding(
    "speechbrain/spkrec-ecapa-voxceleb",
    device=torch.device("cuda"))

from pyannote.audio import Audio
from pyannote.core import Segment

import wave
import contextlib

from sklearn.cluster import AgglomerativeClustering
import numpy as np

/usr/local/lib/python3.10/dist-packages/pyannote/audio/core/io.py:43: UserWarning: torchaudio._backend.set_audio_backend has been deprecated. With dispatcher enabled, this function is no-op. You can remove the function call.
  torchaudio.set_audio_backend("soundfile")
/usr/local/lib/python3.10/dist-packages/torch_audiomentations/utils/io.py:27: UserWarning: torchaudio._backend.set_audio_backend has been deprecated. With dispatcher enabled, this function is no-op. You can remove the function call.
  torchaudio.set_audio_backend("soundfile")


hyperparams.yaml:   0%|          | 0.00/1.92k [00:00<?, ?B/s]

embedding_model.ckpt:   0%|          | 0.00/83.3M [00:00<?, ?B/s]

mean_var_norm_emb.ckpt:   0%|          | 0.00/1.92k [00:00<?, ?B/s]

classifier.ckpt:   0%|          | 0.00/5.53M [00:00<?, ?B/s]

label_encoder.txt:   0%|          | 0.00/129k [00:00<?, ?B/s]

#Распознаем голоса и составляем диалог в формате txt и его превращаем в pdf и сохраняем в Data так же нужно установить шрифт, для корректного отображеня PDF файла. Готовый PDF будет на гит хабе, чтобы не занимать память графического процессора в колабе,можно этот шаг пропустить предварительно загрузив в data PDF файлы, который нам понадобиться при анализе текста, в гит хабе будут готовый обработанный диалог в PDF и один большой PDF файл, кодекс РФ

##Дальше снизу идет код по обработке аудиофайла

In [21]:
from google.colab import files
from fpdf import FPDF
!pip install reportlab

from reportlab.lib.pagesizes import letter
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
from reportlab.platypus import SimpleDocTemplate, Paragraph

# Регистрация пользовательского шрифта
from reportlab.pdfbase import pdfmetrics
from reportlab.pdfbase.ttfonts import TTFont

# model_size: в зависимости от выбранной вами версии 'tiny', 'base', 'small', 'medium', 'large', мы получим разное качество распознавания

In [29]:
num_speakers = 2 #@param {type:"integer"}

language = 'any' #@param ['any', 'English']

model_size = 'large' #@param ['tiny', 'base', 'small', 'medium', 'large']


model_name = model_size
if language == 'English' and model_size != 'large':
  model_name += '.en'

model = whisper.load_model(model_size)

100%|█████████████████████████████████████| 2.88G/2.88G [00:39<00:00, 77.8MiB/s]


#Загружаем аудио и обрабатываем
можем этот шаг пропустить,и загрузить вручную. Так памяти будет больше для анализа

In [30]:
# upload audio file
uploaded = files.upload()
path = next(iter(uploaded))

if path[-3:] != 'wav':
  subprocess.call(['ffmpeg', '-i', path, 'audio.wav', '-y'])
  path = 'audio.wav'

result = model.transcribe(path)
segments = result["segments"]
with contextlib.closing(wave.open(path,'r')) as f:
  frames = f.getnframes()
  rate = f.getframerate()
  duration = frames / float(rate)
audio = Audio()

def segment_embedding(segment):
  start = segment["start"]
  # Whisper overshoots the end timestamp in the last segment
  end = min(duration, segment["end"])
  clip = Segment(start, end)
  waveform, sample_rate = audio.crop(path, clip)
  return embedding_model(waveform[None])

embeddings = np.zeros(shape=(len(segments), 192))
for i, segment in enumerate(segments):
  embeddings[i] = segment_embedding(segment)

embeddings = np.nan_to_num(embeddings)

clustering = AgglomerativeClustering(num_speakers).fit(embeddings)
labels = clustering.labels_
for i in range(len(segments)):
  segments[i]["speaker"] = 'SPEAKER ' + str(labels[i] + 1)

def time(secs):
  return datetime.timedelta(seconds=round(secs))

f = open("transcript.txt", "w")

for (i, segment) in enumerate(segments):
  if i == 0 or segments[i - 1]["speaker"] != segment["speaker"]:
    f.write("\n" + segment["speaker"] + ' ' + str(time(segment["start"])) + '\n')
  f.write(segment["text"][1:] + ' ')
f.close()



##КОнвертируем в PDF##################################################

#Установи шриф(в гит хабе естғ фаЙл)
font_path = '/content/DejaVuSans.ttf'
pdfmetrics.registerFont(TTFont("DejaVu", font_path))

# Создание стилей для абзацев
styles = getSampleStyleSheet()
custom_style = ParagraphStyle(
    'CustomStyle',
    parent=styles['Normal'],
    fontName='DejaVu',
    fontSize=12,
)

def create_pdf(input_file, output_file):
    # Создание PDF-документа
    pdf = SimpleDocTemplate(output_file, pagesize=letter)

    # Открытие текстового файла в режиме чтения
    with open(input_file, "r", encoding='utf-8') as file:
        # Сборка Paragraphs для вставки в документ
        paragraphs = []
        for line in file:
            paragraphs.append(Paragraph(line, custom_style))

    # Добавление Paragraphs в документ
    pdf.build(paragraphs)

# Конвертация текстового файла в PDF
input_file = "transcript.txt"
output_file = "/content/sample_data/Data/transcript_reportlab.pdf"
create_pdf(input_file, output_file)



Saving AUD-20231127-WA0003 to AUD-20231127-WA0003


#Отсөда начинает Анализ текста с PDF фалов, которые лежат в папке Data :/content/sample_data/Data

#Работ по тексту, все файлы PDF в папке Data(их может быть несколько) обрабатываются и по этим файлам бот находит ответы.

In [27]:
loader = PyPDFDirectoryLoader("/content/sample_data/Data")
data = loader.load()

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

#Step 05: Split the Extracted Data into Text Chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=5000, chunk_overlap=100)
text_chunks = text_splitter.split_documents(data)
len(text_chunks)
text_chunks[0]

vector_store = FAISS.from_documents(text_chunks, embedding=embeddings)

In [28]:
text_chunks[0]

Document(page_content='SPEAKER 1 0:00:00  \nАлло, здравствуйте. Пожалуйста, у меня такой вопрос. На сайте я обнаружил \nинформацию, что по условию семейной ипотека можно в том числе взять \nквартиру от физического лица по договору переуступки. Или я неправильно \nпонял информацию?  \nSPEAKER 2 0:00:15  \nКак мы его к вам обращаются?  \nSPEAKER 1 0:00:17  \nЭльдар.  \nSPEAKER 2 0:00:20  \nЭльдар, правильно услышала?  \nSPEAKER 1 0:00:23  \nЭльдар, да. Эльдар.  \nSPEAKER 2 0:00:26  \nДа, у нас можно будет приобрести от физического лица по семейной ипотек е \nв том случае, если он сам ранее приобретал от юридического лица.  \nSPEAKER 1 0:00:33  \nТо есть, грубо говоря, в Авито, которое продает объявления, есть \nобъявления от физлиц, они пишут, что семейная ипотека работает. \nДействительно, она будет работать, даже  если фактически я буду выходить \nна сделку именно с физлицом.  \nSPEAKER 2 0:00:49  \nДа, все верно. Просто главное, чтобы это физическое лицо раньше у Юра \nприобретало, и 

#тут без цикла,вводим запрос в переменную query

In [33]:
qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=vector_store.as_retriever(search_kwargs={"k": 2}))
query = "О чем бы разговор?"


In [34]:
qa.run(query)

Llama.generate: prefix-match hit


' The conversation is about whether it is possible to buy a flat from a physical person using a family mortgage.'

#тут с циклом для несколько вопросов

In [26]:
import sys
#load pdf files
loader = PyPDFDirectoryLoader("/content/sample_data/Data")
data = loader.load()

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
#Step 05: Split the Extracted Data into Text Chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=5000, chunk_overlap=100)
text_chunks = text_splitter.split_documents(data)
len(text_chunks)
text_chunks[0]

vector_store = FAISS.from_documents(text_chunks, embedding=embeddings)
qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=vector_store.as_retriever(search_kwargs={"k": 2}))

while True:
  user_input = input(f"Input Prompt: ")
  if user_input == 'exit':
    print('Exiting')
    sys.exit()
  if user_input == '':
    continue

  result = qa({'query': user_input})

  print(f"Answer: {result['result']}")

Input Prompt: Я убил человека. Что мне будет согласно уголовному кодексу Российской федерации?


/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
Llama.generate: prefix-match hit


Answer:  The answer to your question is not available in the provided context.
Input Prompt: Я убил человека. Что мне будет согласно уголовному кодексу Российской федерации?


Llama.generate: prefix-match hit


Answer:  According to the Russian Federation Criminal Code, the punishment for murder is a sentence of life imprisonment without parole.
Input Prompt: Мой друг оказался сооучастником преступления, что ему за это будет?


...

Answer:  I don't know.
Input Prompt: Кто такой подстрекатель?


Llama.generate: prefix-match hit


Answer:  I don't know
Input Prompt: Подстрекателем признается лицо,


Llama.generate: prefix-match hit


KeyboardInterrupt: 

#РАБОТАЛ ТОЛЬКО С @bot.message_handler(content_types=['text']) чтобы проверить, но увы, почему то вылетает...Так и не смог реализовать это все в тг...Ниже весь ТГ код

In [ ]:
promt_template = "Ты мультилингвальный дружелюбный помощник, который всё знает. Выполни, что тебя просят. Если тебя попросят ответ, но ты его не знаешь, то ответ что не знаешь."

In [ ]:
import tempfile
import os
from pydub import AudioSegment

bot = telebot.TeleBot('6507009266:AAGOY56MPIsTP7qOj8rJcAKT1a1m1WID4WI')#api telegram bot

@bot.message_handler(commands=['start'])
def start(message):
    bot.send_message(message.chat.id, 'Привет! Можешь спрашивать меня! Что тебя интересует?')

@bot.message_handler(content_types=['text'])
def get_text_messages(message):
    user_input = promt_template + message.text # Добавляем сообщение в промпт
    bot.send_message(message.from_user.id, "Я обрабатываю ваш запрос. Ожидайте 2-5 минут")
    result = qa({'query': user_input}) # Ответ модели
    #prediction = GoogleTranslator(source='auto', target='ru').translate(getResponse_(message.text))
    #message = getResponse(message.text)
    bot.send_message(message.from_user.id, result['result'])

@bot.message_handler(content_types=['voice'])
def handle_voice(message):
    voice = message.voice

    # Получаем информацию о голосовом сообщении
    file_id = voice.file_id
    file_info = bot.get_file(file_id)
    downloaded_file = bot.download_file(file_info.file_path)

    temp_audio_wav = tempfile.NamedTemporaryFile(delete=False, suffix='.wav')
    temp_audio_ogg = tempfile.NamedTemporaryFile(delete=False)
    temp_audio_ogg.write(downloaded_file)
    temp_audio_ogg.close()

    # Конвертируем аудиофайл из формата OGG в WAV с помощью pydub
    audio = AudioSegment.from_file(temp_audio_ogg.name, format="ogg")
    audio.export(temp_audio_wav.name, format="wav")

    audio_file = open(temp_audio_wav.name, "rb")
    transcript = client.audio.transcriptions.create(
    model="whisper-1",
    file=audio_file,
    response_format="text"
    )
    user_input = prompt_template.format(text_input=transcript) # Добавляем сообщение в промпт
    bot.send_message(message.from_user.id, "Я обрабатываю ваш запрос. Ожидайте 2-5 минут")
    result = qa({'query': user_input}) # Ответ модели
    #prediction = GoogleTranslator(source='auto', target='ru').translate(getResponse_(message.text))
    #message = getResponse(message.text)
    bot.send_message(message.from_user.id, result)


@bot.message_handler(content_types=['photo'])
def get_photo_messages(message):
    # Обрабатываем полученное изображение
    file_id = message.photo[-1].file_id
    file_info = bot.get_file(file_id)
    downloaded_file = bot.download_file(file_info.file_path)
    # Преобразование байтов изображения в объект Image
    #image = Image.open(io.BytesIO(downloaded_file))
    #buff = io.BytesIO()
    #downloaded_file.save(buff, format="JPEG")
    #img_str = base64.b64encode(buff.getvalue())
    #(width, height) = image.size
    #print((width, height))
    encoded_data = base64.b64encode(downloaded_file).decode("utf-8")
    caption = message.caption
    if caption:
      predict = Response(encoded_data, caption)
    else:
      predict = Response(encoded_data)
    # Отображение изображения
    #plt.imshow(image)
    #plt.axis('off')  # Отключение осей координат
    #plt.show()
    # Здесь можно выполнить дополнительные действия с загруженным изображением
    # Например, сохранить его локально или выполнить обработку

    # Отправляем сообщение пользователю
    bot.send_message(message.from_user.id, predict)

bot.polling(none_stop=True, interval=0)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
Llama.generate: prefix-match hit
